# Fine Tune a model with the Trainer API

In [12]:
# !pip install datasets
# !pip install transformers[torch]
# !pip install accelerate -U
# !pip install evaluate

## Preprocess

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
raw_datasets = load_dataset('glue', 'mrpc')
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
def tokenize(example):
    return tokenizer(example['sentence1'], example['sentence2'], truncation=True)

In [6]:
tokenized_datasets = raw_datasets.map(tokenize, batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [7]:
data_collator = DataCollatorWithPadding(tokenizer)

## Training

In [8]:
from transformers import TrainingArguments, AutoModelForSequenceClassification

In [9]:
training_args = TrainingArguments("test-trainer")

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [14]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.525100
1000,0.310900


TrainOutput(global_step=1377, training_loss=0.3504779070464268, metrics={'train_runtime': 124.9905, 'train_samples_per_second': 88.039, 'train_steps_per_second': 11.017, 'total_flos': 405324636337200.0, 'train_loss': 0.3504779070464268, 'epoch': 3.0})

## Evaluation

In [15]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [16]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [17]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8455882352941176, 'f1': 0.8923076923076922}

In [18]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.513420,0.769608,0.849840
2,0.577200,0.571676,0.813725,0.876623
3,0.411100,0.553644,0.833333,0.881119


TrainOutput(global_step=1377, training_loss=0.44013401383321704, metrics={'train_runtime': 193.8798, 'train_samples_per_second': 56.757, 'train_steps_per_second': 7.102, 'total_flos': 405540469624800.0, 'train_loss': 0.44013401383321704, 'epoch': 3.0})